# Module 2 - Programming Assignment

## Directions

1. Change the name of this file to be your JHED id as in `jsmith299.ipynb`. Because sure you use your JHED ID (it's made out of your name and not your student id which is just letters and numbers).
2. Make sure the notebook you submit is cleanly and fully executed. I do not grade unexecuted notebooks.
3. Submit your notebook back in Blackboard where you downloaded this file.

*Provide the output **exactly** as requested*

In [1]:
from pprint import pprint

## Local Search - Genetic Algorithm

There are some key ideas in the Genetic Algorithm.

First, there is a problem of some kind that either *is* an optimization problem or the solution can be expressed in terms of an optimization problem.
For example, if we wanted to minimize the function

$$f(x) = \sum (x_i - 0.5)^2$$

where $n = 10$.
This *is* an optimization problem. Normally, optimization problems are much, much harder.

![Eggholder](http://www.sfu.ca/~ssurjano/egg.png)!

The function we wish to optimize is often called the **objective function**.
The objective function is closely related to the **fitness** function in the GA.
If we have a **maximization** problem, then we can use the objective function directly as a fitness function.
If we have a **minimization** problem, then we need to convert the objective function into a suitable fitness function, since fitness functions must always mean "more is better".

Second, we need to *encode* candidate solutions using an "alphabet" analogous to G, A, T, C in DNA.
This encoding can be quite abstract.
You saw this in the Self Check.
There a floating point number was encoded as bits, just as in a computer and a sophisticated decoding scheme was then required.

Sometimes, the encoding need not be very complicated at all.
For example, in the real-valued GA, discussed in the Lectures, we could represent 2.73 as....2.73.
This is similarly true for a string matching problem.
We *could* encode "a" as "a", 97, or '01100001'.
And then "hello" would be:

```
["h", "e", "l", "l", "o"]
```

or

```
[104, 101, 108, 108, 111]
```

or

```
0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1
```

In Genetics terminology, this is the **chromosome** of the individual. And if this individual had the **phenotype** "h" for the first character then they would have the **genotype** for "h" (either as "h", 104, or 01101000).

To keep it straight, think **geno**type is **genes** and **pheno**type is **phenomenon**, the actual thing that the genes express.
So while we might encode a number as 10110110 (genotype), the number itself, 182, is what goes into the fitness function.
The environment operates on zebras, not the genes for stripes.

## String Matching

You are going to write a Genetic Algorithm that will solve the problem of matching a target string (at least at the start).
Now, this is kind of silly because in order for this to work, you need to know the target string and if you know the target string, why are you trying to do it?
Well, the problem is *pedagogical*.
It's a fun way of visualizing the GA at work, because as the GA finds better and better candidates, they make more and more sense.

Now, string matching is not *directly* an optimization problem so this falls under the general category of "if we convert the problem into an optimization problem we can solve it with an optimization algorithm" approach to problem solving.
This happens all the time.
We have a problem.
We can't solve it.
We convert it to a problem we *can* solve.
In this case, we're using the GA to solve the optimization part.

And all we need is some sort of measure of the difference between two strings.
We can use that measure as a **loss function**.
A loss function gives us a score tells us how similar two strings are.
The loss function becomes our objective function and we use the GA to minimize it by converting the objective function to a fitness function.
So that's the first step, come up with the loss/objective function.
The only stipulation is that it must calculate the score based on element to element (character to character) comparisons with no global transformations of the candidate or target strings.

And since this is a GA, we need a **genotype**.
The genotype for this problem is a list of "characters" (individual letters aren't special in Python like they are in some other languages):

```
["h", "e", "l", "l", "o"]
```

and the **phenotype** is the resulting string:

```
"hello"
```

In addition to the generic code and problem specific loss function, you'll need to pick parameters for the run.
These parameters include:

1. population size
2. number of generations
3. probability of crossover
4. probability of mutation

You will also need to pick a selection algorithm, either roulette wheel or tournament selection.
In the later case, you will need a tournament size.
This is all part of the problem.

Every **ten** (10) generations, you should print out the fitness, genotype, and phenotype of the best individual in the population for the specific generation.
The function should return the best individual *of the entire run*, using the same format.

In [2]:
ALPHABET = "abcdefghijklmnopqrstuvwxyz "

## Begin with some basics

### `genotype()`
We have a natural mapping in the ALPHABET provided, this is because each character has a position within the ALPHABET; 'a' has the position 0, while ' ' has the position 26. The genotype will therefore be defined as the element wise position of each character of the population member within ALPHABET. Therefore, the string 'abcde' will have the genotype [0,1,2,3,4].

In order to implement `genotype()` I will simply loop through the string passed and use the `str.find()` method which returns the position of letter $i$ within ALPHABET. I then append that position to a list. Once the loop is completed, the function returns the list.

In [3]:
from random import randint, uniform
def genotype(string: str, ALPHABET: str) -> list:
    string_list = [i for i in string]
    genotype = []
    for i in string_list:
        genotype.append(ALPHABET.find(i))
    return genotype

In [4]:
assert genotype('abcde', ALPHABET) == [0,1,2,3,4]
assert genotype('mynameisjose', ALPHABET) == [12, 24, 13, 0, 12, 4, 8, 18, 9, 14, 18, 4]
assert genotype('this is so much fun', ALPHABET) == [19, 7, 8, 18, 26, 8, 18, 26, 18, 14, 26, 12, 20, 2, 7, 26, 5, 20, 13]

### `phenotype()`
Provided that the genotype is just a mapping from the position to the character within ALPHABET, I will use such mapping to get back the string we are looking for. Therefore the phenotype for [0,1,2,3,4] is 'abcde'.
The function `phenotype` below will take in the list `genotype` and the string `ALPHABET` and will loop through the elements of `genotype` using $i$. For each $i$ element of genotype, the function will apend into a list the object in `ALPHABET` in position `i`. Finally, the string elements of the resulting list will be concatenated into a single string to be returned.
Two cells below you will find the translations from `genotype` to `phenotype`. 

In [5]:
def phenotype(genotype: list, ALPHABET: str) -> str:
    phenotype = []
    for i in genotype:
        phenotype.append(ALPHABET[i])
    phenotype = ''.join(phenotype)
    return phenotype

In [6]:
assert phenotype([0,1,2,3,4], ALPHABET) == 'abcde'
assert phenotype([12, 24, 13, 0, 12, 4, 8, 18, 9, 14, 18, 4], ALPHABET) == 'mynameisjose'
assert phenotype([19, 7, 8, 18, 26, 8, 18, 26, 18, 14, 26, 12, 20, 2, 7, 26, 5, 20, 13], ALPHABET) == 'this is so much fun'

### Difference functions
#### `difference_fitness()`
The most basic representation for the fitness score would be to count how many of the genes match in the genotype and in the goal. The following are important considerations:
- Both the  genotype to be evaluated and the genotype of the goal should be of the same length, in the case that they are not of the same length, I have decided to apply the largest penalty. The largest penalty would be that all chracters are different. This would ensure this particular genotype to be unsuitable for selection in the future as it will go down further in the population ranking. 
- Since we are looking to minimize the difference, we need to apply a couple of transformations to the provided score:
 1. I will provide a score that is a ratio of the overall total possible difference. That is, our difference will be expressed as a percentage of the worst case scenario. 
 2. Once I have a value for the difference as a ratio, I will do the transformation:
 $$\text{fitness_value} = \frac{1}{1+\text{difference_ratio}}$$
 Under these two conditions we know that $0.5\leq\text{fitness_value}\leq 1$. This function range should be useful in decreasing the number of generations needed as the ranking should be rather more dramatic than having an unbounded range.

The function below therefore computes the `goal_genotype` as the genotype for the goal, creates a `max_difference` as the number of elements in `goal_genotype`. Then it checks if both genotypes are of the same length. If they are not, the function returns the maximum `fitness_value`. Otherwise the function compares element-wise the genotypes and counts how many are different. Then it resutns the value $\text{fitness_value} = \frac{1}{1+\text{difference_ratio}}$

In [7]:
def difference_fitness(genotype_,goal,ALPHABET):
    goal_genotype = genotype(goal,ALPHABET)
    max_difference = len(goal_genotype)
    if len(genotype_) != len(goal_genotype):
        difference = 1
        return 1 / (1+difference)
    difference = 0
    for i in range(len(genotype_)):
        if goal_genotype[i] != genotype_[i]:
            difference += 1    
    difference_ratio = difference / max_difference
    fitness_value = 1 / (1+difference_ratio)
    return fitness_value

In [8]:
goal = 'this is so much fun'
genotype_ = genotype('mynameisjose',ALPHABET)
# this result should be 0.5 as the strings have different lengths
assert difference_fitness(genotype_,goal,ALPHABET) == 0.5

genotype_ = genotype('this is so much fun',ALPHABET)
# this result should be 1 as the strings are the same
assert difference_fitness(genotype_,goal,ALPHABET) == 1

genotype_ = genotype('ahis is so much fub',ALPHABET)
# We have two character differences, therefore the result should be 1/(1+(2/19)) = 0.9047619047619049
assert difference_fitness(genotype_,goal,ALPHABET) == 0.9047619047619049

#### `reverse_fitness()`
`reverse_fitness()` uses the exact same logic as `difference_fitness()`, the main difference is that `goal_genotype` to be used corresponds to the reversal of the initial `goal_genotype` as computed in line 2 of the function. This means that we are decoding a string provided in reverse by using the genotype element by element. In order to reverse the initial goal genotype we use a list comprehension using a reverse index. Keep in mind that we are not reversing the string but the genotype corresponding to the goal. Also, there is an element wise comparison on the cost function. This corresponds to the instructions of Problem 2.

In [9]:
def reverse_fitness(genotype_,goal,ALPHABET):
    goal_genotype = genotype(goal,ALPHABET)
    goal_genotype = [goal_genotype[i] for i in range(len(goal_genotype)-1,-1,-1)]
    max_difference = len(goal_genotype)
    if len(genotype_) != len(goal_genotype):
        difference = 1
        return 1 / (1+difference)
    difference = 0
    for i in range(len(genotype_)):
        if goal_genotype[i] != genotype_[i]:
            difference += 1
    difference = difference / max_difference
    fitness_value = 1 / (1+difference)
    return fitness_value

In [10]:
goal = 'mynameisjose'
reverse_goal = ''.join([goal[i] for i in range(len(goal)-1,-1,-1)])
genotype_ = genotype(goal,ALPHABET)
# we are checking wether 'mynameisjose' corresponds to its reverse in 'reverse_goal'.
# Since they are the same, it should return 1
assert reverse_fitness(genotype_,reverse_goal,ALPHABET) == 1

goal = "nuf hcum os si siht"
reverse_goal = ''.join([goal[i] for i in range(len(goal)-1,-1,-1)])
genotype_ = genotype(goal,ALPHABET)
# we are checking wether "nuf hcum os si siht" corresponds to its reverse in 'reverse_goal'.
# Since they are the same, it should return 1
# This corresponds to a check on problem 2
assert reverse_fitness(genotype_,reverse_goal,ALPHABET) == 1

goal = "nuf hcum os si siht"
reverse_goal = ''.join([goal[i] for i in range(len(goal)-1,-1,-1)])
goal = "auf hcum os si siha"
genotype_ = genotype(goal,ALPHABET)
# We have two character differences, therefore the result should be 1/(1+(2/19)) = 0.9047619047619049
assert reverse_fitness(genotype_,reverse_goal,ALPHABET) == 0.9047619047619049

#### `rotate_fitness()`
`rotate_fitness()` uses the exact same logic as `difference_fitness()`, the main difference is that `goal_genotype` to be used corresponds to the rotation of the initial `goal_genotype` as computed in line 1 of the function. This means that we are decoding a string provided in a rotation by using the genotype element by element. In order to rotate the initial goal genotype we use a list comprehension which finds the 13th element ahead in the alphabet within the goal phenotype. Keep in mind that we are not rotating the string but the genotype corresponding to the goal. Also, there is an element wise comparison on the cost function. This corresponds to the instructions of Problem 3.

In [11]:
def rotate_fitness(genotype_,goal,ALPHABET):
    goal_genotype = [(ALPHABET.find(x)+13) % len(ALPHABET) for x in goal]
    max_difference = len(goal_genotype)
    if len(genotype_) != len(goal_genotype):
        difference = 1
        return 1 / (1+difference)
    difference = 0
    for i in range(len(genotype_)):
        if goal_genotype[i] != genotype_[i]:
            difference += 1
    difference = difference / max_difference
    fitness_value = 1 / (1+difference)
    return fitness_value

In [12]:
goal = 'thisissomuchfun'
alphabet = "abcdefghijklmnopqrstuvwxyz"
rotated_goal_genotype = [(alphabet.find(x)+13) % len(alphabet) for x in goal]
# Created a rotate version of the genotype to test that it works properly. This should return a 1
assert rotate_fitness(rotated_goal_genotype,goal,alphabet) == 1

goal = 'mynameisjose'
alphabet = "abcdefghijklmnopqrstuvwxyz"
rotated_goal_genotype = [(alphabet.find(x)+13) % len(alphabet) for x in goal]
# Created a rotate version of the genotype to test that it works properly. This should return a 1
assert rotate_fitness(rotated_goal_genotype,goal,alphabet) == 1

## Creating and increasing the population
I define a member of the population as a tuple containing that member's (`fitness_value`,`genotype`,`phenotype`). The population is then formed when one or more members are stored in a sorted list of members. The sorting 
 1. The `create_member()` function takes 

In [13]:
def create_member(random_genotype, ALPHABET: str, goal: str, fitness_function: object) ->tuple:
    genotype_ = random_genotype
    phenotype_ = ''.join([ALPHABET[i] for i in genotype_])
    fitness_value = fitness_function(genotype_,goal,ALPHABET)
    return (fitness_value, genotype_, phenotype_)    

def increase_population(member:tuple, population:list):
    if len(population)==0:
        population.append(member)
        return None
    for i in range(len(population)):
        if member[0] >= population[i][0]:
            population.insert(i, member)
            return None
    population.append(member)
    return None    

def build_population(random_genotypes, ALPHABET, goal, fitness_function):
    population = list()
    i = 0
    for genotype in random_genotypes:
        member = create_member(genotype, ALPHABET, goal, fitness_function)
        increase_population(member, population)
    return population

In [14]:
alphabet = 'abcde'
goal = 'abc'
random_genotypes =[[2,1,2],[0,1,4],[0,2,1],[3,3,4],[0,1,2]]
# Here we force a non-random list of genotypes. We want to see that:
# 1. The population is ordered by fitness_value in descending order
# 2. The first member should be the perfect match [0,1,2]
# 3. The information is stored as needed
assert build_population(random_genotypes, alphabet, goal, difference_fitness) == [(1.0, [0, 1, 2], 'abc'),
                                                                                  (0.75, [0, 1, 4], 'abe'),
                                                                                  (0.75, [2, 1, 2], 'cbc'),
                                                                                  (0.6000000000000001, [0, 2, 1], 'acb'),
                                                                                  (0.5, [3, 3, 4], 'dde')]

## Choosing parents
In order to choose parents I implement a tournament. For this purpose I created two particular functions:
1. The `pick_parent()` function creates a tournament sub_population. The arguments include `choices` corresponding to a list of random choices from the population, and the `population` from which we are choosing the parents. 
2. `pick_parents()` is a wrapper function which selects a tuple of parents (`parent_1`,`parent_2`) while using `pick_parent()` from step one above. The function has as arguments the `population` from which the choices are made and the random `choices` to be implemented. 

In [15]:
def pick_parent(population:list, choices:list)->tuple:
    sub_population = []
    for choice in choices:
        increase_population(population[choice], sub_population)
    return sub_population[0]

def pick_parents(population:list, choices: list) -> tuple:
    parent_1 = pick_parent(population, choices[0])
    parent_2 = pick_parent(population, choices[1])
    return parent_1, parent_2

In [16]:
sample_population = [(1.0, [0, 1, 2], 'abc'),
                     (0.75, [0, 1, 4], 'abe'),
                     (0.75, [2, 1, 2], 'cbc'),
                     (0.6000000000000001, [0, 2, 1], 'acb'),
                     (0.5, [3, 3, 4], 'dde')]
choices = [[0,1],[3,4]]
# Using the previous population, we will force the following choices:
# 1. For the first parent choose the perfect matching member, it should be 'abc'
# 2. For the second member, it should be the better fitted member between members 3 and 4. It should return 'acb'
assert pick_parents(sample_population, choices) == ((1.0, [0, 1, 2], 'abc'), (0.6000000000000001, [0, 2, 1], 'acb'))

## Reproduction, crossover, and mutation
1. The function `crossover()` produces a tuple of genotypes for the children. It takes the parents and based on a random integer `gen_location` selects for `genotype_c1` = `genotype_p1[:gen_location]`+`genotype_p2[gen_location:]` and for `genotype_c2` = `parent_2[:genotype_p2]`+`parent_1[genotype_p1:]`. `genotype_p1` and `genotype_p2` correspond to the genotype locations for the member parents, respectively. 
2. `mutate()` performs the mutation operation. It uses the `randint()` method from the `random` library to generate a `random_location` and a `random_symbol`. With these two, `mutate()` changes the `random_location` of `genotype` for the `random_symbol`. The changes are executed in place.
3. `reproduce` is a wrapper function which reproduces the parents based on `prob_reproduction`,`prob_mutation`, `parent_1`,`parent_2`, `ALPHABET`, `goal`, `fitness_function`. The last three parameters are used to generate a child tuple. Using the `uniform()` method of the `random` package, we generate a uniformly distributed probability. In the case that the probability is less than `prob_reproduction` then the function returns the tuple 0,0. This indicates that there will not be reproduction. In the case that the probability is greater than `prob_reproduction`, the function continues and invokes `crossover()` which returns a tuple of genotypes. A second test is created to see if a uniform probability is greater than `prob_mutation`. If so, `mutate()` is called on both genotypes. Finally, `child_1` and `child_2` are generated by calling the rest of the parameters along with their invocations. `child_1` and `child_2` are returned as a tuple. 

In [17]:
def cross_over(parent_1, parent_2):
    gen_location = randint(0,len(parent_1)-1)
    genotype_p1 = parent_1[1]
    genotype_p2 = parent_2[1]
    genotype_c1 = genotype_p1[:gen_location]
    genotype_c1.extend(genotype_p2[gen_location:])
    genotype_c2 = genotype_p2[:gen_location]
    genotype_c2.extend(genotype_p1[gen_location:])
    return genotype_c1,genotype_c2

def mutate(genotype_,ALPHABET):    
    random_location = randint(0,len(genotype_)-1)
    random_symbol = randint(0,len(ALPHABET)-1)
    genotype_[random_location] = random_symbol

def reproduce(prob_reproduction, prob_mutation, parent_1, parent_2, ALPHABET, goal, fitness_function):
    if uniform(0,1) < prob_reproduction:
        return 0,0
    genotype_c1, genotype_c2 = cross_over(parent_1, parent_2)
    if uniform(0,1) > prob_mutation:
        mutate(genotype_c1, ALPHABET)
        mutate(genotype_c2, ALPHABET)
    child_1 = (fitness_function(genotype_c1,goal,ALPHABET), genotype_c1, ''.join([ALPHABET[i] for i in genotype_c1]))
    child_2 = (fitness_function(genotype_c2,goal,ALPHABET), genotype_c2, ''.join([ALPHABET[i] for i in genotype_c2]))
    return child_1, child_2   

In [43]:
parent_1,parent_2 = (1.0, [0, 1, 2], 'abc'), (0.6000000000000001, [0, 2, 1], 'acb')
# these tests will not try to produce a certain value or values, since there is randomness included in the functions.
# instead we would like to make sure that certain properties are maintained.

# does crossover generate valid genotypes? 
genotype_c1,genotype_c2 = cross_over(parent_1,parent_2)
# 1. a genotype should have the same length of the parents.
assert len(genotype_c1) == len(parent_1[1]) == len(parent_2[1])

# does mutate create a valid mutation?
# a mutation should have no more than one difference
genotype_c1_copy = genotype_c1.copy()
mutate(genotype_c1_copy,alphabet)
differences = 0
assert sum([genotype_c1[i]!=genotype_c1_copy[i] for i in range(len(genotype_c1_copy))])<= 1

# reproduce produce a valid couple of children?
child_1, child_2 = reproduce(0.5, 0.5, parent_1, parent_2, alphabet, goal, difference_fitness)
# child 1 is a tuple
assert isinstance(child_1,tuple)
# child_1[0] is the fitness value and should be a float
assert isinstance(child_1[0],float)
# child_1[1] is the genotype and should be a list
assert isinstance(child_1[1],list)
# child_1[2] is the phenotype and should be a string
assert isinstance(child_1[2],str)
# the phenotype should be contained in the alphabet
for i in child_1[2]:
    assert i in alphabet

## `genetic_algorithm()`
The function `genetic_algorithm()` follows very closely the pseudocode provided in the lectures. There are two differences:
1. I evaluate the fitness of the population when the population is created instead of in a single step.
2. I keep the population sorted by ascending fitness and any new member added to the population is inserted according to that hierarchy.
3. I use a tournament in which I utilize the functions I created to create and maintain populations.

The code will be tested in problems 1,2, and 3.

In [19]:

def genetic_algorithm(N,
                      generations,
                      prob_reproduction,
                      prob_mutation,
                      ALPHABET,
                      goal,
                      fitness_function):
    random_genotypes = [[randint(0,len(ALPHABET)-1) for i in goal] for i in range(N)]
    population = build_population(random_genotypes, ALPHABET, goal, fitness_function)
    g = 1
    while g < generations:
        next_population = list()
        for n in range(N//2):
            choices = [[randint(0,len(population)-1) for i in range(7)] for i in range(2)]
            parent_1, parent_2 = pick_parents(population, choices)
            child_1, child_2 = reproduce(prob_reproduction,
                                         prob_mutation,
                                         parent_1, parent_2,
                                         ALPHABET,
                                         goal,
                                         fitness_function)
            increase_population(child_1, next_population) if child_1 != 0 else increase_population(parent_1, next_population)
            increase_population(child_2, next_population) if child_2 != 0 else increase_population(parent_2, next_population)
                
        population = next_population
        if g%10 == 0:
            print(f'Generation:{g}, {population[0][2]}')  
        g += 1
        if population[0][0] == 1:
            print(f'Generation:{g}, {population[0][2]}')
            return population[0]
                      
    return population[0]

## Problem 1

The target is the string "this is so much fun".
The challenge, aside from implementing the basic algorithm, is deriving a fitness function based on "b" - "p" (for example).
The fitness function should come up with a fitness score based on element to element comparisons between target v. phenotype.

In [20]:
target1 = "this is so much fun"

In [21]:
# set up if you need it.

In [22]:
result1 = genetic_algorithm(N=1000,
                            generations=100,
                            prob_reproduction=0.5,
                            prob_mutation=0.5,
                            ALPHABET=ALPHABET,
                            goal=target1,
                            fitness_function=difference_fitness)

Generation:10, this ikxsovglchbdud
Generation:20, this ig so glch fud
Generation:30, this ig so mlch fun
Generation:40, this is so much fun
Generation:41, this is so much fun


In [23]:
pprint(result1, compact=True)

(1.0, [19, 7, 8, 18, 26, 8, 18, 26, 18, 14, 26, 12, 20, 2, 7, 26, 5, 20, 13],
 'this is so much fun')


## Problem 2

You should have working code now.
The goal here is to think a bit more about fitness functions.
The target string is now, 'nuf hcum os si siht'.
This is obviously target #1 but reversed.
If we just wanted to match the string, this would be trivial.
Instead, this problem, we want to "decode" the string so that the best individual displays the target forwards.
In order to do this, you'll need to come up with a fitness function that measures how successful candidates are towards this goal.
The constraint is that you may not perform any global operations on the target or individuals.
Your fitness function must still compare a single gene against a single gene.
Your solution will likely not be Pythonic but use indexing.
That's ok.
<div style="background: lemonchiffon; margin:20px; padding: 20px;">
    <strong>Important</strong>
    <p>
        You may not reverse an entire string (either target or candidate) at any time.
        Everything must be a computation of one gene against one gene (one letter against one letter).
        Failure to follow these directions will result in 0 points for the problem.
    </p>
</div>

The best individual in the population is the one who expresses this string *forwards*.

In [24]:
target2 = "nuf hcum os si siht"

In [25]:
# set up if you need it.

In [26]:
result2 = genetic_algorithm(N=1000,
                            generations=100,
                            prob_reproduction=0.5,
                            prob_mutation=0.5,
                            ALPHABET=ALPHABET,
                            goal=target2,
                            fitness_function=reverse_fitness)

Generation:10, thmsssbrsoumach viv
Generation:20, thmssib soumuchhvin
Generation:30, this ib soumuch yin
Generation:40, thisois soxmuch fun
Generation:48, this is so much fun


In [27]:
pprint(result2, compact=True)

(1.0, [19, 7, 8, 18, 26, 8, 18, 26, 18, 14, 26, 12, 20, 2, 7, 26, 5, 20, 13],
 'this is so much fun')


## Problem 3

This is a variation on the theme of Problem 2.
The Caeser Cypher replaces each letter of a string with the letter 13 characters down alphabet (rotating from "z" back to "a" as needed).
This is also known as ROT13 (for "rotate 13").
Latin did not have spaces (and the space is not continguous with the letters a-z) so we'll remove them from our alphabet.
Again, the goal is to derive a fitness function that compares a single gene against a single gene, without global transformations.
This fitness function assigns higher scores to individuals that correctly decode the target.

<div style="background: lemonchiffon; margin:20px; padding: 20px;">
    <strong>Important</strong>
    <p>
        You may not apply ROT13 to an entire string (either target or candidate) at any time.
        Everything must be a computation of one gene against one gene.
        Failure to follow these directions will result in 0 points for the problem.
    </p>
</div>

The best individual will express the target *decoded*.

In [28]:
ALPHABET3 = "abcdefghijklmnopqrstuvwxyz"

In [29]:
target3 = "guvfvffbzhpusha"

In [30]:
# set up if you need it

In [31]:
result3 = genetic_algorithm(N=1000,
                            generations=100,
                            prob_reproduction=0.5,
                            prob_mutation=0.5,
                            ALPHABET=ALPHABET3,
                            goal=target3,
                            fitness_function=rotate_fitness)

Generation:10, thgsiqtlmkchcun
Generation:20, thgsiqsamuchfun
Generation:30, thgsissomuchfon
Generation:34, thisissomuchfun


In [32]:
pprint(result3, compact=True)

(1.0, [19, 7, 8, 18, 8, 18, 18, 14, 12, 20, 2, 7, 5, 20, 13], 'thisissomuchfun')


## Problem 4

There is no code for this problem.

In Problem 3, we assumed we knew what the shift was in ROT-13.
What if we didn't?
Describe how you might solve that problem including a description of the solution encoding (chromosome and interpretation) and fitness function. Assume we can add spaces into the message.

**answer here**
Suppose we do not know the actual shift as we did in this case. However we do know that there is a level of rotation and we also know that there is a particular string that it should decode to. We can therefore create a fitness function that searches for the level of shift within the alphabet that correspond to the decryption. This can be implemented by some of the local search methods we have learned through this course or simply by brute force (provided the alphabet is small). This would probably be an intermediate step within the fitness function. After, we can proceed with the genetic algorithm. This new fitness function would however be able to handle all different types of rotations within the alphabet. 

## Challenge

**You do not need to do this problem and it won't be graded if you do. It's just here if you want to push your understanding.**

The original GA used binary encodings for everything.
We're basically using a Base 27 encoding.
You could, however, write a version of the algorithm that uses an 8 bit encoding for each letter (ignore spaces as they're a bit of a bother).
That is, a 4 letter candidate looks like this:

```
0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1
```

If you wrote your `genetic_algorithm` code general enough, with higher order functions, you should be able to implement it using bit strings instead of latin strings.

## Before You Submit...

1. Did you provide output exactly as requested?
2. Did you re-execute the entire notebook? ("Restart Kernel and Rull All Cells...")
3. If you did not complete the assignment or had difficulty please explain what gave you the most difficulty in the Markdown cell below.
4. Did you change the name of the file to `jhed_id.ipynb`?

Do not submit any other files.